<a href="https://colab.research.google.com/github/hajri94/research-project/blob/main/Test_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd
import datetime as dt 
import matplotlib.pyplot as plt
from numpy import percentile


In [44]:
# We download the excel file
from google.colab import files
uploaded = files.upload()

Saving Test dataframe 10259.xlsx to Test dataframe 10259 (2).xlsx


In [ ]:
# Convert the excel file into pandas Dataframe
import io 
df = pd.read_excel(io.BytesIO(uploaded['Test dataframe 10259.xlsx']))
# We round the columns (POWER , Temperature, Irradiation.flux) to one decimal place.
df['Temperature']=df['Temperature'].round(1)
df['Irradiation.flux']=df['Irradiation.flux'].round(1)
df




In [70]:
# We create multiple POWER lagged columns (we choose 12 lagged dependent variables  ) 
lagged1 = pd.concat([df['POWER'].shift(lag).rename('{}_lag{}'.format(df['POWER'].name, lag)) for lag in range(13)], axis=1)
lagged1.dropna().astype(int)





,POWER_lag0,POWER_lag1,POWER_lag2,POWER_lag3,POWER_lag4,POWER_lag5,POWER_lag6,POWER_lag7,POWER_lag8,POWER_lag9,POWER_lag10,POWER_lag11,POWER_lag12
12,19,19,19,17,18,18,17,18,18,21,18,18,23
13,18,19,19,19,17,18,18,17,18,18,21,18,18
14,17,18,19,19,19,17,18,18,17,18,18,21,18
15,18,17,18,19,19,19,17,18,18,17,18,18,21
16,17,18,17,18,19,19,19,17,18,18,17,18,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,18,18,21,19,16,17,15,16,17,16,16,16,17
8756,17,18,18,21,19,16,17,15,16,17,16,16,16
8757,22,17,18,18,21,19,16,17,15,16,17,16,16
8758,17,22,17,18,18,21,19,16,17,15,16,17,16


In [ ]:
# We create multiple Temperature lagged columns (we choose 24 lagged independent variables)
lagged2 = pd.concat([df['Temperature'].shift(lag).rename('{}_lag{}'.format(df['Temperature'].name, lag)) for lag in range(25)], axis=1)
lagged2.dropna().astype(int)

In [ ]:
# We create multiple Irradiation.flux lagged columns(we choose 24 lagged independent variables)
lagged3 = pd.concat([df['Irradiation.flux'].shift(lag).rename('{}_lag{}'.format(df['Irradiation.flux'].name, lag)) for lag in range(25)], axis=1)
lagged1.dropna().astype(int)


In [ ]:
# Maximize the size of Column in google colab 
#(we resize the maximum number of columns in google colab to fit our dataframe)
from google.colab.data_table import DataTable
DataTable.max_columns = 300
# Joining the DataFrame of POWER Lagged Columns to the Dataframe of Temperature Lagged Columns 
# We use the concat function to join 2 dataframes lagged1 (POWER) + lagged2 (Temperature) => Creation of new dataframe called "result".
result = pd.concat([lagged1,lagged2], axis=1, join='inner')
result.dropna().astype(int)

In [ ]:
#Joining the DataFrame of POWER and Temperature Lagged Columns (POWER+Temperature=> result dataframe) to the Dataframe of Irradiation flux Lagged Columns 
# We use the concat function to join 2 dataframes result (POWER+Temperature) + lagged3 (Irradiation flux)  => Creation of new dataframe called "result1"
result1 = pd.concat([result,lagged3], axis=1, join='inner')
r1=result1.dropna().astype(int)
r1

In [ ]:
# We join the "Result1" Dataframe (Power , Temperature, Irradiation flux ) to the original dataframe (the one that we have uploaded from excel)
# we perform this line of code in order to link the two columns (READ_DATE),(Week_Day) to the datframe (result2). 
# The new dataframe need to be more organized as we see some columns that are repeated. 
result2 = pd.concat([r1,df], axis=1, join='inner')
#we remove the columns titled (POWER , Temperature, Irradiation flux)
dk=result2.drop(['POWER', 'Temperature','Irradiation.flux'], axis = 1) 
# To move the Read_Date and Week Date columns to first Position we implement some initial step: 
# First we remove the targeted column "Week_Day" from its intial position in the dataframe.  
first_column = dk.pop('Week_Day')
# Second we insert the removed column "Week_Day" to first position.
dk.insert(0, 'Week_Day', first_column)
# We perform the previous step for the "READ_DATE" column.
second_column = dk.pop('READ_DATE')
dk.insert(0, 'READ_DATE', second_column)
dk

In [57]:
# After creating the new dataframe (called dk), we link it to calendar data table  which is put in an excel file to carry out this step :  
# We download the calendar data excel file
from google.colab import files
uploaded = files.upload()

Saving calendar Data .xlsx to calendar Data  (2).xlsx


In [79]:
# Convert the excel file into Dataframe
import io 
dm = pd.read_excel(io.BytesIO(uploaded['calendar Data .xlsx']))
#In the "Bank Holidays" column we replace the NaN value with zeros.
dm['Bank Holidays'] = dm['Bank Holidays'].fillna(0)
# we remove the "READ_DATE" and "Week_Day" columns to avoid the Repetition of these column when linking it to the dataframe "Dk" 
remove_column = dm.pop('READ_DATE')
remove2_column = dm.pop('Week_Day')


In [82]:
dm.dtypes
dm['Bank Holidays'] = dm['Bank Holidays'].astype(int)

In [ ]:
# we join the the calendar data dataframe uploaded from excel file to the dataframe "Dk" 
result3 = pd.concat([dk,dm], axis=1, join='inner')
result3


In [ ]:
#we move the postion of  "Hour_day" column to first position .
column = result3.pop('Hour_day')
result3.insert(0, 'Hour_day', column)
# We drop the nan value from the dataframe
r3=result3.dropna()
r3




In [36]:
# With this code we can perform the fitering of each of the "Hour_day" value.  
HourWeek=result3[result3['Hour_day']== 1]

In [91]:
#Separation of indeppendent variables and dependent variables

import pandas as pd
import statsmodels.api as sm
X = r3[['POWER_lag1','POWER_lag2','POWER_lag3','POWER_lag4','POWER_lag5','POWER_lag6','POWER_lag7','POWER_lag8','POWER_lag9','POWER_lag10','POWER_lag11','POWER_lag12',
        'Temperature_lag0','Temperature_lag1','Temperature_lag2','Temperature_lag3','Temperature_lag4','Temperature_lag5','Temperature_lag6','Temperature_lag7','Temperature_lag8',
        'Temperature_lag9','Temperature_lag10','Temperature_lag11','Temperature_lag12','Temperature_lag13','Temperature_lag14','Temperature_lag15','Temperature_lag16','Temperature_lag17','Temperature_lag18','Temperature_lag19','Temperature_lag20','Temperature_lag21','Temperature_lag22','Temperature_lag23','Temperature_lag24',
        'Irradiation.flux_lag0','Irradiation.flux_lag1','Irradiation.flux_lag2','Irradiation.flux_lag3','Irradiation.flux_lag4','Irradiation.flux_lag5','Irradiation.flux_lag6','Irradiation.flux_lag7','Irradiation.flux_lag8','Irradiation.flux_lag9','Irradiation.flux_lag10','Irradiation.flux_lag11','Irradiation.flux_lag12','Irradiation.flux_lag13','Irradiation.flux_lag14','Irradiation.flux_lag15','Irradiation.flux_lag16','Irradiation.flux_lag17','Irradiation.flux_lag18','Irradiation.flux_lag19','Irradiation.flux_lag20','Irradiation.flux_lag21','Irradiation.flux_lag22','Irradiation.flux_lag23',
        'Irradiation.flux_lag24','Tuesday 0h','Tuesday 1h','Tuesday 2h','Tuesday 3h','Tuesday 4h','Tuesday 5h','Tuesday 6h','Tuesday 7h','Tuesday 8h','Tuesday 9h','Tuesday 10h','Tuesday 11h','Tuesday 12h','Tuesday 13h','Tuesday 14h',
        'Tuesday 15h','Tuesday 16h','Tuesday 17h','Tuesday 18h','Tuesday 19h','Tuesday 20h','Tuesday 21h','Tuesday 22h','Tuesday 23h','Wednesday 0h','Wednesday 1h','Wednesday 2h','Wednesday 3h','Wednesday 4h','Wednesday 5h','Wednesday 6h','Wednesday 7h','Wednesday 8h','Wednesday 9h','Wednesday 10h','Wednesday 11h','Wednesday 12h','Wednesday 13h','Wednesday 14h',
        'Wednesday 15h','Wednesday 16h','Wednesday 17h','Wednesday 18h','Wednesday 19h','Wednesday 20h','Wednesday 21h','Wednesday 22h','Wednesday 23h','Thursday 0h','Thursday 1h','Thursday 2h','Thursday 3h','Thursday 4h','Thursday 5h','Thursday 6h','Thursday 7h','Thursday 8h','Thursday 9h','Thursday 10h','Thursday 11h','Thursday 12h','Thursday 13h','Thursday 14h',
        'Thursday 15h','Thursday 16h','Thursday 17h','Thursday 18h','Thursday 19h','Thursday 20h','Thursday 21h','Thursday 22h','Thursday 23h','Friday 0h','Friday 1h','Friday 2h','Friday 3h','Friday 4h','Friday 5h','Friday 6h','Friday 7h','Friday 8h','Friday 9h','Friday 10h','Friday 11h','Friday 12h','Friday 13h','Friday 14h',
        'Friday 15h','Friday 16h','Friday 17h','Friday 18h','Friday 19h','Friday 20h','Friday 21h','Friday 22h','Friday 23h','Saturday 0h','Saturday 1h','Saturday 2h','Saturday 3h','Saturday 4h','Saturday 5h','Saturday 6h','Saturday 7h','Saturday 8h','Saturday 9h','Saturday 10h','Saturday 11h','Saturday 12h','Saturday 13h','Saturday 14h',
        'Saturday 15h','Saturday 16h','Saturday 17h','Saturday 18h','Saturday 19h','Saturday 20h','Saturday 21h','Saturday 22h','Saturday 23h','Sunday 0h','Sunday 1h','Sunday 2h','Sunday 3h','Sunday 4h','Sunday 5h','Sunday 6h','Sunday 7h','Sunday 8h','Sunday 9h','Sunday 10h','Sunday 11h','Sunday 12h','Sunday 13h','Sunday 14h',
        'Sunday 15h','Sunday 16h','Sunday 17h','Sunday 18h','Sunday 19h','Sunday 20h','Sunday 21h','Sunday 22h','Sunday 23h','Monday 0h','Monday 1h','Monday 2h','Monday 3h','Monday 4h','Monday 5h','Monday 6h','Monday 7h','Monday 8h','Monday 9h','Monday 10h','Monday 11h','Monday 12h','Monday 13h','Monday 14h',
        'Monday 15h','Monday 16h','Monday 17h','Monday 18h','Monday 19h','Monday 20h','Monday 21h','Monday 22h','Monday 23h','Bank Holidays']] 
Y = r3['POWER_lag0']



In [92]:
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X, Y)


LinearRegression()

In [93]:
# with statsmodels
X = sm.add_constant(X) # adding a constant
 
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:             POWER_lag0   R-squared:                       0.939
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     549.0
Date:                Thu, 28 Apr 2022   Prob (F-statistic):               0.00
Time:                        11:49:55   Log-Likelihood:                -17991.
No. Observations:                8404   AIC:                         3.644e+04
Df Residuals:                    8173   BIC:                         3.807e+04
Df Model:                         230                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      3